In [3]:
import os
import re
import module_traitement as m

# Processing subtitles

This notebook is used to get one sentence per subtitles in the .vtt files from the **Matignon_LSF** dataset.

## Chargement des dossiers

In [2]:
# TEST -  Mediapi - LSF 
file_with_path = m.lister_fichiers_with_path("../data/aligned_mediapi/")
folder = m.lister_fichiers("../data/aligned_mediapi/")
output_seg = "test_new_seg_mediapi/"
output_cleaning = "test_cleaning_mediapi/"

In [66]:
last_one = ["1afbd17607.vtt","3094d986b3.vtt","92e68e4fcd.vtt"]
file_with_path = ["../data/aligned_mediapi/1afbd17607.vtt","../data/aligned_mediapi/3094d986b3.vtt","../data/aligned_mediapi/92e68e4fcd.vtt"]
folder = ["../data/aligned_mediapi/1afbd17607.vtt","../data/aligned_mediapi/3094d986b3.vtt","../data/aligned_mediapi/92e68e4fcd.vtt"]
output_seg = "last_seg/"
output_cleaning = "last_cleaning"

# Files normalisation

Cleaning the files with regexp (rule-based) to make it easier to have the "sentence per subtitles" files.

In [2]:
def convertir_grand_nombre(nombre_texte):
    nombre_sans_points = re.sub(r'(\d)\.(\d)', r'\1\2', nombre_texte)
    return nombre_sans_points

def remplacer_points_adresses_email(texte):
    # regexp for email
    pattern = r'\b[\w.%+-]+@[\w.-]+\.[a-zA-Z]{2,}\b'

    # replace "." with "POINT"
    def remplacer(match):
        return match.group().replace('.', 'POINT')

    texte_modifie = re.sub(pattern, remplacer, texte)
    return texte_modifie

def remplacer_points_adresses(texte):
    # regexp for website
    pattern = r'\b(?:https?://)?(?:www\.)?[\w.-]+\.[a-zA-Z]{2,}\b'

    # Fonction de remplacement pour remplacer les points par "POINT"
    def remplacer(match):
        return match.group().replace('.', 'POINT')

    texte_modifie = re.sub(pattern, remplacer, texte)
    return texte_modifie

def normaliser_points_de_suspension(texte):
    #add a space after "..."
    texte_modifie = re.sub(r'\.\.\.(\w)', r'... \1', texte)

    # Remove a space before ellipses preceded by a letter.
    texte_modifie = re.sub(r'(\w)\s*\.\.\.', r'\1...', texte_modifie)

    return texte_modifie

def remplacer_ponctuation_html(texte):
    substitutions = [
        ("(", "&#40;"),
        (")", "&#41;"),
        ("?", "&#63;"),
        ("!", "&#33;"),
        (".", "&#46;"),
        ("...", "&#8230;")
    ]

    def remplacer_match(match):
        contenu_parentheses = match.group(1)
        for recherche, remplacement in substitutions:
            contenu_parentheses = contenu_parentheses.replace(recherche, remplacement)
        return f'({contenu_parentheses})'

    texte_modifie = re.sub(r'\(([^)]*)\)', remplacer_match, texte)
    return texte_modifie

def get_dict_vtt_clean(input):
    with open(input,encoding="utf-8") as f:
        lines = f.readlines()

    dict_sub = {}
    i = 0
    j = 0  

    while j < len(lines): 
        element = lines[j]
        if element.startswith("00:") or element.startswith("01:") or element.startswith("02:"):
            # Extract start and end time
            timing_line = element.strip().split(' --> ')
            start_time, end_time = timing_line

            text = ""
            while j + 1 < len(lines) and not lines[j + 1].startswith("00:") and not lines[j+1].startswith("01:") and not lines[j+1].startswith("02:"):
                j += 1
                content = lines[j]
                text = text + " " + content.strip()
                text = normaliser_points_de_suspension(text)
                text = text.replace("(...)","[SUSPENSIONP]") #régler ce problème.
                text = text.replace("....","[POINTS4]")
                text = remplacer_ponctuation_html(text)
                ### changement
                text = text.replace("…","...")
                text = text.replace("...","[SUSPENSION].") ### gérer la double suspension !
                text = re.sub(r'["“”«»]', '', text)
                text = text.replace("(???)","[INTERROGATION3]")
                text = text.replace("?!","[INTEREXCL].")
                text = text.replace("(?)","[INTERROGATION1]")
                text = text.replace("(??)","[INTERROGATION2]")
                text = text.replace("( ?)","[INTERROGATION1]")
                text = text.replace("?,","[INTERROGATION],")
                text = text.replace("!,","[EXCLAMATION],")
                text = text.replace("!.","[EXCLAMPOINT],")
                text = text.replace("etc.,","[ETC],")
                text = text.replace("etc.)","[ETC])")
                text = text.replace("etc. .","[ETC].")
                text = text.replace("Etc","etc")
                text = text.replace("PAM !","[PAM]")
                text = text.replace("Média’Pi","Média'Pi")
                text = text.replace("Média'Pi!","[NOM_MEDIA]")
                text = text.replace("Média'Pi !","[NOM_MEDIA]")
                text = text.replace("Media'Pi !","[NOM_MEDIA]")
                text = text.replace("Média'Pi&nbsp;!","[NOM_MEDIA]")
                text = text.replace("Média' Pi !","[NOM_MEDIA]")
                
                text = text.replace(".e.s","")
                text = text.replace(".ne.s","")
                text = text.replace(".e.","")
                text = text.replace(".e","")
                text = text.replace("!!","!")
                text = text.replace("??","?")
                text = remplacer_points_adresses_email(text)
                text = remplacer_points_adresses(text)
                text = text.replace(".,","[POINT],")
                text = text.replace("y.a","y a")
                ### 
                text=text.replace("... -G. Attal : ","")
                text=text.replace("-G. Attal : ","")
                text=text.replace("G. Attal : ","")
                text = text.replace("-Bonjour", "Bonjour")
                text = text.replace("Bonjour.", "Bonjour,")
                text = text.replace(" M."," Monsieur")
                text = text.replace(" Mme"," Madame")
                text = text.replace("-"," ")
                text = convertir_grand_nombre(text)
                

            dict_sub[i] = {'start': start_time, 'end': end_time, 'text': text.strip()}
            i += 1

        j += 1

    return dict_sub



def remplacer_entites_html(chaine):
    substitutions = [
    ("(", "&#40;"),
    (")", "&#41;"),
    ("?", "&#63;"),
    ("!", "&#33;"),
    (".", "&#46;"),
    ("...", "&#8230;")
]
    for new, old in substitutions:
        chaine = chaine.replace(old, new)
    return chaine

def reverse_cleaning(input):
    with open(input,encoding="utf-8") as f:
        lines = f.readlines()

    dict_sub = {}
    i = 0
    j = 0  

    while j < len(lines): 
        element = lines[j]
        if element.startswith("00:") or element.startswith("01:") or element.startswith("02:"):
            #Extract end and start time 
            timing_line = element.strip().split(' --> ')
            start_time, end_time = timing_line

            text = ""
            while j + 1 < len(lines) and not lines[j + 1].startswith("00:") and not lines[j+1].startswith("01:") and not lines[j+1].startswith("02:"):
                j += 1
                content = lines[j]
                text = text + " " + content.strip()
                text = text.replace("[POINT],",".,")
                text = text.replace("[SUSPENSIONP]","(...)") 
                text = text.replace("[POINTS4]","....")
                text = text.replace("…","...")
                text = text.replace("[INTERROGATION3]","(???)")
                text = text.replace("[INTEREXCL].","?!")
                text = text.replace("[INTERROGATION1]","(?)")
                text = text.replace("[INTERROGATION2]","(??)")
                text = text.replace("[INTERROGATION],","?,")
                text = text.replace("[EXCLAMATION],","!,")
                text = text.replace("[EXCLAMPOINT],","!.")
                text = text.replace("[ETC],","etc.,")
                text = text.replace("[ETC])","etc.)")
                text = text.replace("[ETC].","etc.")
                text = text.replace("[PAM]","PAM !")
                text = text.replace("[NOM_MEDIA]","Média'Pi !")
                text = text.replace("[SUSPENSION].","...") 
                text = text.replace("POINT",".")
                text = remplacer_entites_html(text)
                

            dict_sub[i] = {'start': start_time, 'end': end_time, 'text': text.strip()}
            i += 1

        j += 1

    return dict_sub



In [4]:
for file,name in zip(file_with_path,folder):
    dict_sub = get_dict_vtt_clean(file)
    m.create_vtt_file(dict_sub,f"{output_cleaning}/{name}")

# Get a sentence file

Obtain a sentence file (from the clean data) - .txt

In [5]:
file_with_path = m.lister_fichiers_with_path("test_cleaning_mediapi")
folder = m.lister_fichiers("test_cleaning_mediapi")
output_sent = "test_sent_file_mediapi"

In [6]:
pattern = re.compile(r' [A-Z]\.')
for file,name in zip(file_with_path,folder):
    dict_sub = get_dict_vtt_clean(file)
    new_text = ""
    for k,v in dict_sub.items():
        for kk,vv in v.items():
            if kk == "text":
                new_text = new_text + vv.strip() + " "
    phrases = m.segmenter_texte_en_phrases(new_text)
    with open(f"{output_sent}/{name}", 'w', encoding='utf-8') as f:
        for index, element in enumerate(phrases):
            f.write(element.strip())
            if index < len(phrases) - 1:
                f.write("\n")


# Get an oversegmented subtitle's file (.vtt)

From the clean data, we segment our clean subtitle's file (.vtt) following the strong punctuation. We allow the over segmentation, because we can concatenate the subunite after this processing operation.

In [7]:
file_with_path = m.lister_fichiers_with_path("test_cleaning_mediapi")
folder = m.lister_fichiers("test_cleaning_mediapi")
output_seg = "test_new_seg_mediapi/"

In [8]:
for file,name in zip(file_with_path,folder):
    print(f"TRAITEMENT {file} ---- {name}")
    dict_sub = get_dict_vtt_clean(file)
    new_dict = {}
    mm = 0
    pattern = r'([.!?]+)[^)]' # good solution
    sous_unite = []
    for k, v in dict_sub.items():
        for kk, vv in v.items():
            if kk == "text":
                # Replace the point between two capital letters with '#'
                modified_text = re.sub(r'(?<=[A-Z])\.(?=[A-Z])', '#', vv) #attention à les supprimer pour remettre les points à la place à la fin
                # Use re.split() to split the text based on the pattern
                sentences = re.split(pattern, modified_text)
                # Combine pairs of adjacent list elements (sentence + punctuation)
                result = [sentences[i] + sentences[i + 1] if i < len(sentences) - 1 else sentences[i] for i in range(0, len(sentences), 2)]
                # Remove empty strings from the result
                result = [sentence.strip() for sentence in result if sentence.strip()]
                if len(result) == 1:
                    if mm not in new_dict:
                        new_dict[mm]=v
                        mm = mm +1
                else:
                    if result:
                        print(f"resultat : {result}")
                        start_time_str = v["start"]
                        end_time_str = v["end"]
                        # start_time = m.conv_str_to_time(start_time_str)
                        # end_time = m.conv_str_to_time(end_time_str)
                        nb_of_carach = len(v["text"])
                        duration = m.time_to_seconds(end_time_str) - m.time_to_seconds(start_time_str)
                        duration_sec = duration
                        print(duration)
                        sec_par_letter = duration_sec / nb_of_carach
                        for match in result:
                            len_match = len(match)
                            duration_match = len_match*sec_par_letter
                            if mm not in new_dict:
                                #print(end_time)
                                end_time = m.ajouter_secondes(start_time_str,duration_match)
                                print(end_time)
                                print(f"start time ({type(start_time_str)}) : {start_time_str}, end time ({type(end_time)}) : {end_time}, text : {match}")
                                new_dict[mm]={'start':start_time_str,"end":end_time,'text':match}
                                start_time_str = end_time
                                mm = mm +1
                    else:
                        continue
    m.create_vtt_file(new_dict,f"{output_seg}/{name}")

                    
    

TRAITEMENT test_cleaning_mediapi/b7f2d8f0c3.vtt ---- b7f2d8f0c3.vtt
TRAITEMENT test_cleaning_mediapi/3d0b82b459.vtt ---- 3d0b82b459.vtt
resultat : ["Est ce qu'il se livrera à un massacre ?", 'Mystère[SUSPENSION].']
2.920000000000016
00:06:39.398
start time (<class 'str'>) : 00:06:37.500, end time (<class 'str'>) : 00:06:39.398, text : Est ce qu'il se livrera à un massacre ?
00:06:40.371
start time (<class 'str'>) : 00:06:39.398, end time (<class 'str'>) : 00:06:40.371, text : Mystère[SUSPENSION].
TRAITEMENT test_cleaning_mediapi/44f554f914.vtt ---- 44f554f914.vtt
TRAITEMENT test_cleaning_mediapi/b9a51f4361.vtt ---- b9a51f4361.vtt
TRAITEMENT test_cleaning_mediapi/4e073949b1.vtt ---- 4e073949b1.vtt
TRAITEMENT test_cleaning_mediapi/0b1437bc85.vtt ---- 0b1437bc85.vtt
TRAITEMENT test_cleaning_mediapi/752500b761.vtt ---- 752500b761.vtt
TRAITEMENT test_cleaning_mediapi/cba6cefad2.vtt ---- cba6cefad2.vtt
TRAITEMENT test_cleaning_mediapi/41c606553f.vtt ---- 41c606553f.vtt
TRAITEMENT test_cleani

# Concatenate the subunite from the subtitles in sentences

In [9]:
# TEST Mediapi
pre_seg_file = "test_new_seg_mediapi/"
sentence_file = "test_sent_file_mediapi/"
resultat = m.lister_fichiers_with_path(pre_seg_file)
path_sentence = m.lister_fichiers_with_path(sentence_file)
resultat_output = m.lister_fichiers(pre_seg_file)
sentences_only = m.lister_fichiers(sentence_file)
output_folder = "test_sent_seg_mediapi/"

In [ ]:
last_one = ["1afbd17607.vtt","3094d986b3.vtt","92e68e4fcd.vtt"]
file_with_path = ["../data/aligned_mediapi/1afbd17607.vtt","../data/aligned_mediapi/3094d986b3.vtt","../data/aligned_mediapi/92e68e4fcd.vtt"]
folder = ["../data/aligned_mediapi/1afbd17607.vtt","../data/aligned_mediapi/3094d986b3.vtt","../data/aligned_mediapi/92e68e4fcd.vtt"]
output_seg = "last_seg/"
output_cleaning = "last_cleaning"

In [82]:
pre_seg_file = "test_new_seg_mediapi/"
sentence_file = "test_sent_file_mediapi/"
resultat = ["test_new_seg_mediapi/1afbd17607.vtt","test_new_seg_mediapi/3094d986b3.vtt","test_new_seg_mediapi/92e68e4fcd.vtt"]
path_sentence = ["test_sent_file_mediapi/1afbd17607.vtt","test_sent_file_mediapi/3094d986b3.vtt","test_sent_file_mediapi/92e68e4fcd.vtt"]
resultat_output = ["1afbd17607.vtt","3094d986b3.vtt","92e68e4fcd.vtt"]
sentences_only = ["1afbd17607.vtt","3094d986b3.vtt","92e68e4fcd.vtt"]
output_folder = "test_sent_seg_mediapi/"

In [77]:
# Creation of the new subtitle's files.

for sub,sub_name,phr,phr_name in zip(resultat,resultat_output,path_sentence,sentences_only):
    print(f"TRAITEMENT : {sub_name} && {phr_name}")
    print(f"TRAITEMENT : {sub} && {phr}")
    dict_sub = m.get_dict_vtt(sub)
    sentences = m.get_sentences(phr)
    keys = list(dict_sub.keys())
    i = 0
    j = 0
    mm = 0
    new_dict = {}
    content = ""
    while i < len(keys) and j < len(sentences):
        #print(mm)
        key = keys[i]
        value = dict_sub[key]
        unite = value['text'].strip()
        sent = sentences[j].strip()
        #print(f"'{unite} --> '{sent}'")
        # Je regarde si l'unité est dans la phrase. Si elle est dans la phrase, j'ajoute l'information start, et tant que les unités sont dans la même phrase je concatène pour ajouter l'information du texte
        # I check if the unit is in the sentence. If it is, I add the start information, and as long as the units are in the same sentence, I concatenate to add the text information.
        if unite in sent:
            #print(f"'{unite} --> '{sent}'")
            if mm not in new_dict:
                new_dict[mm] = {"start": value['start']}
            if 'text' not in new_dict[mm]:
                new_dict[mm]["text"] = unite.strip()
            else:
                new_dict[mm]['text'] += f" {unite.strip()}" # adding space here, problem to check
            i = i +1

            if i == len(keys):
                new_dict[mm]['end'] = value['end']
        else:
            old_key = keys[i - 1]
            if mm in new_dict and 'end' not in new_dict[mm]:
                new_dict[mm]['end'] = dict_sub[old_key]['end']
                #print(new_dict)
                mm += 1
            j = j +1
    m.create_vtt_file(new_dict,f"{output_folder}/{sub_name}")

TRAITEMENT : 1afbd17607.vtt && 1afbd17607.vtt
TRAITEMENT : test_new_seg_mediapi/1afbd17607.vtt && test_sent_file_mediapi/1afbd17607.vtt
TRAITEMENT : 3094d986b3.vtt && 3094d986b3.vtt
TRAITEMENT : test_new_seg_mediapi/3094d986b3.vtt && test_sent_file_mediapi/3094d986b3.vtt
TRAITEMENT : 92e68e4fcd.vtt && 92e68e4fcd.vtt
TRAITEMENT : test_new_seg_mediapi/92e68e4fcd.vtt && test_sent_file_mediapi/92e68e4fcd.vtt


# An other segmentation if necessary

In [70]:
file_with_path = m.lister_fichiers_with_path("test_sent_seg_mediapi")
folder = m.lister_fichiers("test_sent_seg_mediapi")
output_seg="try_seg_end/"

In [71]:
for file,name in zip(file_with_path,folder):
    print(f"TRAITEMENT {file} ---- {name}")
    dict_sub = get_dict_vtt_clean(file)
    new_dict = {}
    mm = 0
    pattern = r'([.!?]+)[^)]' # bonne solution
    sous_unite = []
    for k, v in dict_sub.items():
        for kk, vv in v.items():
            if kk == "text":
                # Replace the point between two capital letters with '#'
                modified_text = re.sub(r'(?<=[A-Z])\.(?=[A-Z])', '#', vv)
                # Use re.split() to split the text based on the pattern
                sentences = re.split(pattern, modified_text)
                # Combine pairs of adjacent list elements (sentence + punctuation)
                result = [sentences[i] + sentences[i + 1] if i < len(sentences) - 1 else sentences[i] for i in range(0, len(sentences), 2)]
                # Remove empty strings from the result
                result = [sentence.strip() for sentence in result if sentence.strip()]
                if len(result) == 1:
                    if mm not in new_dict:
                        new_dict[mm]=v
                        mm = mm +1
                else:
                    if result:
                        print(f"resultat : {result}")
                        start_time_str = v["start"]
                        end_time_str = v["end"]
                        # start_time = m.conv_str_to_time(start_time_str)
                        # end_time = m.conv_str_to_time(end_time_str)
                        nb_of_carach = len(v["text"])
                        duration = m.time_to_seconds(end_time_str) - m.time_to_seconds(start_time_str)
                        duration_sec = duration
                        print(duration)
                        sec_par_letter = duration_sec / nb_of_carach
                        for match in result:
                            len_match = len(match)
                            duration_match = len_match*sec_par_letter
                            if mm not in new_dict:
                                #print(end_time)
                                end_time = m.ajouter_secondes(start_time_str,duration_match)
                                print(end_time)
                                print(f"start time ({type(start_time_str)}) : {start_time_str}, end time ({type(end_time)}) : {end_time}, text : {match}")
                                new_dict[mm]={'start':start_time_str,"end":end_time,'text':match}
                                start_time_str = end_time
                                mm = mm +1
                    else:
                        continue
    m.create_vtt_file(new_dict,f"{output_seg}/{name}")

                    
    

TRAITEMENT test_sent_seg_mediapi/b7f2d8f0c3.vtt ---- b7f2d8f0c3.vtt
TRAITEMENT test_sent_seg_mediapi/3d0b82b459.vtt ---- 3d0b82b459.vtt
TRAITEMENT test_sent_seg_mediapi/44f554f914.vtt ---- 44f554f914.vtt
TRAITEMENT test_sent_seg_mediapi/b9a51f4361.vtt ---- b9a51f4361.vtt
TRAITEMENT test_sent_seg_mediapi/4e073949b1.vtt ---- 4e073949b1.vtt
TRAITEMENT test_sent_seg_mediapi/0b1437bc85.vtt ---- 0b1437bc85.vtt
TRAITEMENT test_sent_seg_mediapi/752500b761.vtt ---- 752500b761.vtt
TRAITEMENT test_sent_seg_mediapi/cba6cefad2.vtt ---- cba6cefad2.vtt
TRAITEMENT test_sent_seg_mediapi/41c606553f.vtt ---- 41c606553f.vtt
TRAITEMENT test_sent_seg_mediapi/3f1b2118ca.vtt ---- 3f1b2118ca.vtt
TRAITEMENT test_sent_seg_mediapi/ed969c3e70.vtt ---- ed969c3e70.vtt
TRAITEMENT test_sent_seg_mediapi/26f1ff8385.vtt ---- 26f1ff8385.vtt
TRAITEMENT test_sent_seg_mediapi/2b4b33189c.vtt ---- 2b4b33189c.vtt
TRAITEMENT test_sent_seg_mediapi/ada7d10a19.vtt ---- ada7d10a19.vtt
TRAITEMENT test_sent_seg_mediapi/fd41a24117.vtt 

# Manual verification

In [91]:
path_sentence = m.lister_fichiers_with_path("test_sent_file_mediapi/") # vérifier qu'on charge le bon fichier !
output_folder = m.lister_fichiers_with_path("test_sent_seg_mediapi/")
output_folder = sorted(output_folder)
print(output_folder)

['test_sent_seg_mediapi/0004d291e4.vtt', 'test_sent_seg_mediapi/0007d118a9.vtt', 'test_sent_seg_mediapi/00099d279c.vtt', 'test_sent_seg_mediapi/000ad4adb5.vtt', 'test_sent_seg_mediapi/0026226fb1.vtt', 'test_sent_seg_mediapi/005971b87d.vtt', 'test_sent_seg_mediapi/005c1d83b5.vtt', 'test_sent_seg_mediapi/00613c92e3.vtt', 'test_sent_seg_mediapi/006c967985.vtt', 'test_sent_seg_mediapi/007e93fe10.vtt', 'test_sent_seg_mediapi/0083bddf4e.vtt', 'test_sent_seg_mediapi/008420cac0.vtt', 'test_sent_seg_mediapi/00961862b7.vtt', 'test_sent_seg_mediapi/00d88dbb91.vtt', 'test_sent_seg_mediapi/00ef761570.vtt', 'test_sent_seg_mediapi/0115c2b09b.vtt', 'test_sent_seg_mediapi/0126d5ec9a.vtt', 'test_sent_seg_mediapi/012af290a0.vtt', 'test_sent_seg_mediapi/01396e52f9.vtt', 'test_sent_seg_mediapi/013a4d2e94.vtt', 'test_sent_seg_mediapi/01438658d4.vtt', 'test_sent_seg_mediapi/014413bb15.vtt', 'test_sent_seg_mediapi/014465666a.vtt', 'test_sent_seg_mediapi/0152d6f219.vtt', 'test_sent_seg_mediapi/015d410c82.vtt',

In [92]:
path_sentence = sorted(path_sentence)
print(path_sentence)

['test_sent_file_mediapi/0004d291e4.vtt', 'test_sent_file_mediapi/0007d118a9.vtt', 'test_sent_file_mediapi/00099d279c.vtt', 'test_sent_file_mediapi/000ad4adb5.vtt', 'test_sent_file_mediapi/0026226fb1.vtt', 'test_sent_file_mediapi/005971b87d.vtt', 'test_sent_file_mediapi/005c1d83b5.vtt', 'test_sent_file_mediapi/00613c92e3.vtt', 'test_sent_file_mediapi/006c967985.vtt', 'test_sent_file_mediapi/007e93fe10.vtt', 'test_sent_file_mediapi/0083bddf4e.vtt', 'test_sent_file_mediapi/008420cac0.vtt', 'test_sent_file_mediapi/00961862b7.vtt', 'test_sent_file_mediapi/00d88dbb91.vtt', 'test_sent_file_mediapi/00ef761570.vtt', 'test_sent_file_mediapi/0115c2b09b.vtt', 'test_sent_file_mediapi/0126d5ec9a.vtt', 'test_sent_file_mediapi/012af290a0.vtt', 'test_sent_file_mediapi/01396e52f9.vtt', 'test_sent_file_mediapi/013a4d2e94.vtt', 'test_sent_file_mediapi/01438658d4.vtt', 'test_sent_file_mediapi/014413bb15.vtt', 'test_sent_file_mediapi/014465666a.vtt', 'test_sent_file_mediapi/0152d6f219.vtt', 'test_sent_file

In [93]:
def supprimer_derniere_chaine_vide(liste):
    if liste and isinstance(liste[-1], str) and not liste[-1]:
        del liste[-1]
    return liste


In [94]:
nb_file = 0
name_file = []
good_file = []
for sub_file, sent_file in zip(output_folder,path_sentence):
    sub_dict = m.get_dict_vtt(sub_file)
    sentence = m.get_sentences(sent_file)
    new_sent = []
    for k,v in sub_dict.items():
        for kk,vv in v.items():
            if kk == "text":
                if "(... )" in vv:
                    vv = vv.replace("(... )","(...)")
                new_sent.append(vv)
    new_sent = supprimer_derniere_chaine_vide(new_sent)
    differences = m.comparer_listes(new_sent, sentence)
    if differences:
        print(f"TRAITEMENT : {sub_file} && {sent_file}")
        for position, element_a, element_b in differences:
            nb_file = nb_file +1
            print(f"Différence à la position {position}: {element_a} vs {element_b}")
            name_file.append(sub_file)
            break
    else:
        good_file.append(sub_file)

print(f"Nombres de fichiers contenant une erreur : {nb_file}")

TRAITEMENT : test_sent_seg_mediapi/01d57cb44a.vtt && test_sent_file_mediapi/01d57cb44a.vtt
Différence à la position 92: Ce S'News est terminé, notre prochain rendez vous est au mois de décembre ! Ce S'News est terminé, notre prochain rendez vous est au mois de décembre ! vs Ce S'News est terminé, notre prochain rendez vous est au mois de décembre !
TRAITEMENT : test_sent_seg_mediapi/01e9808f72.vtt && test_sent_file_mediapi/01e9808f72.vtt
Différence à la position 1: La marque Yves Saint Laurent l'a choisi pour être son égérie dans un spot publicitaire, pour sa gamme de produits de beauté Nu. Avant d'être lancé, Il s’est d'abord fait connaitre via sa chaîne Youtube et son compte Instagram, dans lesquels il parle de son appartenance à plusieurs minorités : transgenre, genderqueer, sourd, d’origine chinoise et juif. vs La marque Yves Saint Laurent l'a choisi pour être son égérie dans un spot publicitaire, pour sa gamme de produits de beauté Nu.
TRAITEMENT : test_sent_seg_mediapi/024e1e4beb

In [ ]:
last_one = ["1afbd17607.vtt","3094d986b3.vtt","92e68e4fcd.vtt"]

# Reverse cleaning

We used some special token to replace some challenging one from the subtitles. We do a reverse cleaning in order to have the original subtitle's token.

In [12]:
substitutions = [
    ("(", "&#40;"),
    (")", "&#41;"),
    ("?", "&#63;"),
    ("!", "&#33;"),
    (".", "&#46;"),
    ("...", "&#8230;")
]

def remplacer_entites_html(chaine, substitutions):
    for new, old in substitutions:
        chaine = chaine.replace(old, new)
    return chaine

In [7]:
files_with_path = m.lister_fichiers_with_path("gold_mediapi_sent_based")
name = m.lister_fichiers("gold_mediapi_sent_based")


for file,name_file in zip(files_with_path,name):
    dict_sub = reverse_cleaning(file)
    m.create_vtt_file(dict_sub,f"reverse_cleaning_test1/{name_file}")